### Load packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tools.plotting import scatter_matrix
%matplotlib inline
from sklearn import cross_validation as cv
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import cross_validation
from sklearn import preprocessing

### Load csv

In [2]:
motionloc = 'C:/Users/Valued Customer/Desktop/motion.csv'
nomotionloc = 'C:/Users/Valued Customer/Desktop/nomotion.csv'
motion = pd.read_csv(motionloc, header=None,skip_blank_lines=True,skiprows=[22]) # load whole table, row 22 is NA
nomotion = pd.read_csv(nomotionloc, header=None,skip_blank_lines=True)
motion_data = motion.ix[1:,2:6]  # column 2-6 are features, 1st row is column name
nomotion_data = nomotion.ix[1:,2:6]
motion_class = motion.ix[1:,7] # column 7 is class
nomotion_class = nomotion.ix[1:,7]
x=motion_data.append(nomotion_data,ignore_index = True)
x=x.astype('float64') # convert string to float
y=pd.concat([motion_class,nomotion_class],ignore_index = True)
y=y.astype('category') # convert string to categorical

### Exploratory analysis

In [3]:
# data_all=x
# data_all['class']=y

In [4]:
# _=data_all.boxplot(by='class')

In [5]:
# _=data_all.groupby('class').boxplot() 

In [6]:
# _=scatter_matrix(x, alpha=0.2, figsize=(6, 6))#, diagonal='kde')

In [7]:
# _=data_all.hist()

### Split training and testing

In [13]:
x_train, x_test, y_train, y_test=cv.train_test_split(x,y,test_size=0.3)

### Data preprocessing

In [14]:
x_train_scaled = preprocessing.scale(x_train)
scaler = preprocessing.StandardScaler().fit(x_train)
x_test_scaled= scaler.transform(x_test)

In [15]:
data_scaled=pd.DataFrame(x_train_scaled)
data_scaled['class']=y_train

### Build classifier

In [16]:
names=['LDA','QDA','random forest','SVC','KNN']
classifiers=[LinearDiscriminantAnalysis(),
             QuadraticDiscriminantAnalysis(),
             RandomForestClassifier(max_depth=5, n_estimators=3, max_features=1),
             SVC(gamma=2, C=1),
             KNeighborsClassifier(3)]
for name, clf in zip(names,classifiers):
    clf.fit(x_train_scaled,y_train)
    print(clf.score(x_test_scaled,y_test))
    #score = cross_validation.cross_val_score(clf, x_train_scaled, y_train, cv=10)#,scoring=scoring)
    #print'{}: accuracy {}+/-{}'.format(name,score.mean(),score.std())

0.416666666667
0.666666666667
0.5
0.5
0.5
